In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=e47ac1ed14925ba1a715059eebe321e1a89fa50b512d2d260f2f39510ec07e17
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=1c2ce7badc586c1a81833dd073a3c7a61c55fbef49bb1a5d3e05cb4b290caf57
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout
from sklearn.model_selection import train_test_split
from kerastuner import RandomSearch

In [6]:
from google.colab import files
uploded=files.upload()

Saving sensor_readings_24.csv to sensor_readings_24.csv


In [13]:
data =pd.read_csv('sensor_readings_24.csv')

In [14]:
data.head()

,0.438,0.498,3.625,3.645,5.000,2.918,5.000.1,2.351,2.332,2.643,1.698,1.687,1.698.1,1.717,1.744,0.593,0.502,0.493,0.504,0.445,0.431,0.444,0.440,0.429,Slight-Right-Turn
0,0.438,0.498,3.625,3.648,5.0,2.918,5.0,2.637,2.332,2.649,1.695,1.687,1.695,1.720,1.744,0.592,0.502,0.493,0.504,0.449,0.431,0.444,0.443,0.429,Slight-Right-Turn
1,0.438,0.498,3.625,3.629,5.0,2.918,5.0,2.637,2.334,2.643,1.696,1.687,1.695,1.717,1.744,0.593,0.502,0.493,0.504,0.449,0.431,0.444,0.446,0.429,Slight-Right-Turn
2,0.437,0.501,3.625,3.626,5.0,2.918,5.0,2.353,2.334,2.642,1.730,1.687,1.695,1.717,1.744,0.593,0.502,0.493,0.504,0.449,0.431,0.444,0.444,0.429,Slight-Right-Turn
3,0.438,0.498,3.626,3.629,5.0,2.918,5.0,2.640,2.334,2.639,1.696,1.687,1.695,1.717,1.744,0.592,0.502,0.493,0.504,0.449,0.431,0.444,0.441,0.429,Slight-Right-Turn
4,0.439,0.498,3.626,3.629,5.0,2.918,5.0,2.633,2.334,2.645,1.705,1.686,1.694,1.719,1.744,0.589,0.502,0.493,0.504,0.446,0.431,0.444,0.444,0.430,Slight-Right-Turn


In [15]:
data.describe()

,0.438,0.498,3.625,3.645,5.000,2.918,5.000.1,2.351,2.332,2.643,1.698,1.687,1.698.1,1.717,1.744,0.593,0.502,0.493,0.504,0.445,0.431,0.444,0.440,0.429
count,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.00000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000,5455.000000
mean,1.471807,2.327378,2.489139,2.796346,2.958178,2.893068,3.350811,2.540431,3.125767,2.832421,2.549554,2.077848,2.125856,2.190573,2.205857,1.202223,0.989921,0.91035,1.058212,1.076436,1.016030,1.778279,1.555250,1.578719
std,0.802753,1.410058,1.247454,1.309438,1.339062,1.282693,1.413645,1.111653,1.357047,1.307960,1.382107,1.249404,1.407285,1.577004,1.715580,1.098638,0.942138,0.88959,1.144711,1.141570,0.887485,1.571726,1.291478,1.150480
min,0.400000,0.437000,0.470000,0.833000,1.120000,1.114000,1.122000,0.859000,0.836000,0.810000,0.783000,0.778000,0.770000,0.756000,0.495000,0.424000,0.373000,0.35400,0.340000,0.355000,0.380000,0.370000,0.367000,0.377000
25%,0.921000,1.362000,1.538500,1.731000,1.774000,1.785500,1.930500,1.618000,1.799500,1.636000,1.579000,1.293000,1.190500,1.031000,0.860000,0.690500,0.581000,0.53000,0.523500,0.542000,0.567000,0.743000,0.792000,0.884000
50%,1.335000,1.905000,2.064000,2.457000,2.667000,2.682000,3.225000,2.171000,2.802000,2.679000,2.000000,1.689000,1.609000,1.493000,1.328000,0.803000,0.738000,0.68500,0.691000,0.693000,0.764000,1.031000,1.071000,1.289000
75%,1.814000,2.683000,2.739000,4.094000,4.314000,3.835500,5.000000,3.193000,5.000000,3.526500,3.217500,2.283000,2.358500,2.739500,4.436500,1.159000,0.913000,0.83700,0.857000,0.863000,1.002500,2.068500,1.560000,1.657500
max,5.000000,5.025000,5.029000,5.017000,5.000000,5.005000,5.008000,5.087000,5.000000,5.022000,5.019000,5.000000,5.003000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [ ]:
x = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [ ]:
x_train.shape

(3818, 24)

In [ ]:
y_train.shape

(3818,)

In [ ]:
set(y_train)

{'Move-Forward', 'Sharp-Right-Turn', 'Slight-Left-Turn', 'Slight-Right-Turn'}

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_train
y_test = le.fit_transform(y_test)
y_test

array([0, 0, 1, ..., 2, 0, 0])

In [ ]:
set(y_train)

{0, 1, 2, 3}

In [ ]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,4)
y_test = np_utils.to_categorical(y_test,4)

In [ ]:
y_train.shape

(3818, 4)

In [ ]:
def hyper_model(hp):
  model = Sequential()
  model.add(Dense(units=hp.Int('Dense_unit1',min_value=32,max_value=128,step=16),activation=hp.Choice('Dense_act1',values=['sigmoid','relu','tanh']),input_dim=24))
  model.add(Dense(units=hp.Int('Dense_unit2',min_value=32,max_value=128,step=16),activation=hp.Choice('Dense_act2',values=['sigmoid','relu','tanh'])))
  model.add(Dense(units=hp.Int('Dense_unit3',min_value=32,max_value=128,step=16),activation=hp.Choice('Dense_act3',values=['sigmoid','relu','tanh'])))
  model.add(Dense(4,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer=hp.Choice('optimizer',values=['adam','sgd']),metrics=['accuracy'])
  return model

In [ ]:
tuner_search = RandomSearch(hyper_model,objective='val_accuracy',max_trials=4,directory='Hyper_parameter4',project_name='sensor_model')

In [ ]:
tuner_search.search(x_train,y_train, epochs=5,validation_split=0.1,batch_size=100)

Epoch 1/5
35/35 [==============================] - 0s 7ms/step - loss: 1.1153 - accuracy: 0.5122 - val_loss: 0.9284 - val_accuracy: 0.5942
Epoch 2/5
35/35 [==============================] - 0s 5ms/step - loss: 0.8532 - accuracy: 0.6429 - val_loss: 0.7728 - val_accuracy: 0.6649
Epoch 3/5
35/35 [==============================] - 0s 4ms/step - loss: 0.7214 - accuracy: 0.7180 - val_loss: 0.6866 - val_accuracy: 0.7330
Epoch 4/5
35/35 [==============================] - 0s 5ms/step - loss: 0.6416 - accuracy: 0.7541 - val_loss: 0.6229 - val_accuracy: 0.7513
Epoch 5/5
35/35 [==============================] - 0s 4ms/step - loss: 0.5861 - accuracy: 0.7718 - val_loss: 0.5559 - val_accuracy: 0.7827


Epoch 1/5
35/35 [==============================] - 0s 7ms/step - loss: 1.1447 - accuracy: 0.5265 - val_loss: 1.0202 - val_accuracy: 0.5838
Epoch 2/5
35/35 [==============================] - 0s 4ms/step - loss: 0.9361 - accuracy: 0.6027 - val_loss: 0.8658 - val_accuracy: 0.6047
Epoch 3/5
35/35 [==============================] - 0s 4ms/step - loss: 0.8004 - accuracy: 0.6493 - val_loss: 0.7784 - val_accuracy: 0.6571
Epoch 4/5
35/35 [==============================] - 0s 4ms/step - loss: 0.7379 - accuracy: 0.6863 - val_loss: 0.7444 - val_accuracy: 0.6885
Epoch 5/5
35/35 [==============================] - 0s 5ms/step - loss: 0.6937 - accuracy: 0.7008 - val_loss: 0.7047 - val_accuracy: 0.7173


Epoch 1/5
35/35 [==============================] - 0s 7ms/step - loss: 1.1994 - accuracy: 0.4566 - val_loss: 1.1483 - val_accuracy: 0.4921
Epoch 2/5
35/35 [==============================] - 0s 4ms/step - loss: 1.0826 - accuracy: 0.5780 - val_loss: 1.0008 - val_accuracy: 0.6152
Epoch 3/5
35/35 [==============================] - 0s 4ms/step - loss: 0.9488 - accuracy: 0.5995 - val_loss: 0.8941 - val_accuracy: 0.6126
Epoch 4/5
35/35 [==============================] - 0s 4ms/step - loss: 0.8560 - accuracy: 0.6225 - val_loss: 0.8419 - val_accuracy: 0.6204
Epoch 5/5
35/35 [==============================] - 0s 4ms/step - loss: 0.7896 - accuracy: 0.6703 - val_loss: 0.7953 - val_accuracy: 0.6230


Epoch 1/5
35/35 [==============================] - 0s 6ms/step - loss: 1.2212 - accuracy: 0.4182 - val_loss: 1.1877 - val_accuracy: 0.4031
Epoch 2/5
35/35 [==============================] - 0s 4ms/step - loss: 1.1788 - accuracy: 0.4729 - val_loss: 1.1667 - val_accuracy: 0.5131
Epoch 3/5
35/35 [==============================] - 0s 4ms/step - loss: 1.1630 - accuracy: 0.5204 - val_loss: 1.1535 - val_accuracy: 0.5236
Epoch 4/5
35/35 [==============================] - 0s 3ms/step - loss: 1.1508 - accuracy: 0.5393 - val_loss: 1.1447 - val_accuracy: 0.5105
Epoch 5/5
35/35 [==============================] - 0s 4ms/step - loss: 1.1400 - accuracy: 0.5378 - val_loss: 1.1308 - val_accuracy: 0.5314


INFO:tensorflow:Oracle triggered exit


In [ ]:
best_model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
best_model.fit(x_train,y_train, epochs=10,batch_size=100)

Epoch 1/10
39/39 [==============================] - 0s 3ms/step - loss: 0.5326 - accuracy: 0.7928
Epoch 2/10
39/39 [==============================] - 0s 3ms/step - loss: 0.4858 - accuracy: 0.8224
Epoch 3/10
39/39 [==============================] - 0s 3ms/step - loss: 0.4438 - accuracy: 0.8387
Epoch 4/10
39/39 [==============================] - 0s 3ms/step - loss: 0.4081 - accuracy: 0.8570
Epoch 5/10
39/39 [==============================] - 0s 3ms/step - loss: 0.3913 - accuracy: 0.8465
Epoch 6/10
39/39 [==============================] - 0s 3ms/step - loss: 0.3551 - accuracy: 0.8704
Epoch 7/10
39/39 [==============================] - 0s 3ms/step - loss: 0.3456 - accuracy: 0.8761
Epoch 8/10
39/39 [==============================] - 0s 3ms/step - loss: 0.3168 - accuracy: 0.8916
Epoch 9/10
39/39 [==============================] - 0s 3ms/step - loss: 0.2942 - accuracy: 0.8968
Epoch 10/10
39/39 [==============================] - 0s 3ms/step - loss: 0.2855 - accuracy: 0.8968
